# Les méthodes d’évaluation d’un modèle prédictif

Après des heures à paramétrer au mieux un modèle d’apprentissage avec la certitude d’avoir écarté les biais qui pourraient orienter les résultats – rappelons qu’un mauvais modèle peut fournir de très mauvais résultats avec une précision étonnante –, les premières prédictions sortent de la machine et nous souhaitons évaluer leur qualité afin de le passer en production ou non.

Bien entendu, le cas présenté plus haut ne vaut que pour sa généralité ; dans la pratique, les méthodes d’évaluation sont présentes à chaque étape de la programmation d’un modèle si bien que presque aucun choix ne devrait être pris sans validation par une métrique ou une autre. Comme nous nous sommes concentrés sur deux types d’algorithmes, nous n’aborderons que les méthodes d’évaluation pour les tâches de régression et de classification.

## Mesurer une erreur

Par métrique, on entend une façon d’évaluer la qualité d’une prédiction par mesure de la distance entre la réalité observée et la valeur calculée par un algorithme. Si l’on souhaite par exemple prédire la note d’un élève au prochain examen de français en se basant uniquement sur sa moyenne dans la matière – disons 12 –, l’algorithme de prédiction vaudra simplement :

$$\hat{y} = \mu$$

L’élève obtient finalement une note de 11. Pour mesurer l’erreur de la prédiction, il suffit de soustraire $\hat{y}$ de $y$, soit un résultat de $-1$. Remarquons que si sa note avait été de 13, le résultat aurait été positif : $13 - 12 = 1$. Or, $-1$  et $+1$ étant situés à égales distances de la prédiction, ils représentent la même réalité géométrique. Dans les deux cas, l’erreur est réputée être de $1$. On utilise donc plutôt une formule impliquant les valeurs absolues :

$$e = \lvert y - \hat{y}\rvert$$

## Métriques pour les tâches de régression

Dans l’exemple de l’introduction, il s’agissait simplement de calculer l’erreur pour un couple unique prédiction/résultat. Qu’en serait-il si nous avions une série de prédictions et une série de résultats ? Plutôt que de calculer indépendamment les erreurs de chaque prédiction, nous préférerions obtenir une mesure de l’ensemble.

Et pour corser le tout, il existe plusieurs métriques qui ne répondent pas tout à fait aux mêmes enjeux. Choisir la plus adaptée à la situation peut ainsi devenir une nécessité pour ajuster plus finement encore le modèle.

Prenons le cas fictif de la pluviométrie au-dessus de la commune de Pont-Aven avec d’un côté les précipitations mesurées en millimètres pour les mois de janvier à mai 2022 et, de l’autre, des prédictions imaginaires :

In [ ]:
import pandas as pd
import seaborn as sns

# series
series = {
    "months": ["Jan", "Feb", "March", "April", "May"],
    "rainfall": [70, 65, 55, 50, 9],
    "predictions": [35, 60, 75, 45, 20]
}
# dataframe
df = pd.DataFrame(series)

# column 'months' still an id var, while two others are registered in a col 'Measure'
df2 = pd.melt(df, id_vars="months", var_name="Measure", value_name="mm")

# graph
_ = sns.lineplot(data=df2, x="months", y="mm", hue="Measure", marker="o")
sns.despine()

### Le coefficient de détermination linéaire de Pearson ($R^2$)

Le $R^2$ est un score qui mesure de la qualité de la prédiction d’un modèle de régression linéaire en évaluant la variance d’une variable par rapport à une autre variable. Il est défini par la relation suivante pour un résultat généralement compris dans l’intervalle $[0,1]$ :

$$R^2 = 1 - \frac{\sum_{i=1}^k(y_i - \hat{y}_i)^2}{\sum_{i=1}^k(y_i - \bar{y})^2}$$

Son analyse est très intuitive mais elle implique deux critères :
- le modèle est linéaire ;
- une seule variable explicative est concernée.

Un $R^2$ de 1.0 est un score parfait quand un score de 0.0 indiquerait que le modèle prédit toujours la valeur attendue (la moyenne). Un score négatif reste possible mais serait révélateur d’une erreur de méthodologie (données arbitrairement mauvaises).

Dans le cas de notre exemple, la prédiction n’est clairement pas linéaire, aussi le calcul du $R^2$ ne devrait pas servir pour l’évaluation de notre modèle. À titre d’exercice, voyons ce qu’il donne en invoquant la fonction `r2_score()` du module `metrics` de *Scikit-learn* :

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(df.rainfall, df.predictions)
r2

### L’erreur quadratique moyenne (MSE)

La MSE (*mean square error*) et sa cousine, la RMSE (*root mean square error*), sont les deux métriques les plus couramment utilisées en *machine learning*. La MSE calcule la moyenne des carrés des erreurs selon la formule :

$$\text{MSE} = \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2$$

Comme entre en jeu un calcul au carré, la MSE pénalise plus fortement les grandes erreurs et, dans le même ordre d’idée, sera très sensible aux données aberrantes (*outliers*). La fonction dans *Scikit-learn* est `mean_squared_error()` :

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(df.rainfall, df.predictions)
mse

### La racine de l’erreur quadratique moyenne (RMSE)

Plus facile à interpréter que la MSE, la RMSE (*root mean square error*) s’exprime dans l’unité de la variable à prédire en extrayant la racine carrée de la MSE :

$$\text{RMSE} = \sqrt{ \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2 }$$

À noter qu’elle souffre des mêmes limites que la MSE : une grande sensibilité aux *outliers* ainsi qu’une incidence forte sur les erreurs importantes. Pour la calculer avec *Scikit-learn*, il suffit de prendre la racine carrée de la MSE :

In [ ]:
# power of 0.5 = square root
rmse = mse ** 0.5
rmse

### L’erreur absolue moyenne (MAE)

Quand les valeurs extrêmes d’un jeu de données sont quantitativement importantes, la RMSE pourrait conduire à des erreurs d’interprétation. Dans un tel cas de figure, la MAE (*mean absolute error*) peut lui être préférée : en calculant la moyenne de valeurs absolues, elle ne pénalise plus autant les grandes erreurs et se rend moins sensible aux données aberrantes. La formule de la MAE vaut ainsi :

$$\text{MAE} = \frac{1}{k}\sum_{i=0}^{k-1} \lvert y_i - \hat{y}_i\rvert$$

Dans *Scikit-learn*, la fonction `mean_absolute_error()` se charge du calcul :

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(df.rainfall, df.predictions)
mae

## Métriques pour les tâches de classification

Dans la grande famille des tâches de classification, nous reconnaissons trois catégories :
- **la classification binaire** (l’observation appartient-elle à la classe cible ou non ?) ;
- **la classification multi-classes** (parmi toutes, à quelle classe l’observation appartient-elle ?) ;
- **la classification multi-étiquettes** (l’observation appartient-elle à plusieurs classes ?).

Là encore, selon la nature de la tâche, nous ne choisirons pas forcément la même métrique.

### Mesures générales de performance

Évaluer les performances d’un classificateur étant plus complexe que pour les tâches de régression, il est nécessaire d’aborder en premier lieu certaines généralités.

#### L’exactitude (*accuracy*)

La toute première est de ne jamais oublier qu’un très mauvais classificateur peut obtenir des résultats stupéfiants. Prenons l’exemple d’un jeu de données factice qui comporte cent observations étiquettées selon deux modalités : chat ou pas chat.

In [ ]:
from random import shuffle

# 5 out of an hundred are cats
dataset = [
    "cat" if i < 5 else "non-cat"
    for i in range(100)
]
# every day I'm shufflin
shuffle(dataset)

Générons une autre liste pour les prédictions avec la seule étiquette *non-cat* :

In [ ]:
predictions = [ "non-cat" for i in range(100) ]

Et mesurons la performance de notre algorithme prédictif grâce au score d’exactitude (*accuracy*) :

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(dataset, predictions)

**95 % !** Un taux d’exactitude à faire pâlir les diseuses de bonne aventure, non ? Pour cette raison, on ne se satisfera jamais du score d’exactitude, quitte même à s’en méfier dès que les jeux de données sont asymétriques.

#### Une matrice de confusion

La matrice de confusion repose sur un principe simple : compter le nombre de fois où les observations ont été bien ou mal étiquetées. Elle révèle ainsi quatre informations essentielles :
- **les vrais positifs** (*true positive*), le classificateur a repéré qu’il s’agissait d’un chat ;
- **les vrais négatifs** (*true negative*), le classificateur a repéré qu’il ne s’agissait pas d’un chat ;
- **les faux positifs** (*false positive*), le classificateur a cru qu’il s’agissait d’un chat ;
- **les faux négatifs** (*false negative*), le classificateur aurait dû voir qu’il s’agissait d’un chat.

Préparons un jeu de données aléatoire avec la fonction `make_classification()` du module `sklearn.datasets` et effectuons des prédictions à partir d’un modèle de classification naïve bayésienne :

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# make dataset
X, y = make_classification(n_samples=1000, n_features=5, n_informative=3, random_state=42)

# make train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

# model
model = GaussianNB()
model.fit(X_train, y_train)

# predictions
y_pred = model.predict(X_test)

La matrice de confusion s’obtient avec la fonction `confusion_matrix()` du module `sklearn.metrics` :

In [ ]:
from sklearn.metrics import confusion_matrix

cfm = confusion_matrix(y_test, y_pred)

Chaque ligne correspond à une classe réelle et chaque colonne à une classe prédite avec, sur la première ligne, la classe négative et, sur la seconde, la classe positive, tel que dans le tableau suivant :

|prédites/réelles|Classe négative|Classe positive|
|-|:-:|:-:|
|Classe négative|TN (85)|FP (9)|
|Classe positive|FN (3)|TP (103)|

Une méthode `.ravel()` permet de récupérer chacun de ces indicateurs :

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

Notons également l’existence d’une classe `ConfusionMatrixDisplay` pour afficher la matrice de confusion :

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

display = ConfusionMatrixDisplay(confusion_matrix=cfm, display_labels=model.classes_)
_ = display.plot()

En guise de conclusion, signalons que cette matrice de confusion détermine plusieurs scores : la précision, la sensibilité (ou rappel) et le score $F_1$.

### Classification binaire

### Classification multi-classes